In [26]:
f1_txt=""


filepath = "../dune/01 dune.txt"
with open(filepath,encoding="utf8", errors='ignore') as fp:
   line = fp.readline()
   cnt = 1
   while line:
       #print("{}".format( line.strip()))
       f1_txt=f1_txt+line
       line = fp.readline()
       cnt += 1
       

In [18]:
print(f1_txt[:100])